In [1]:
from tqdm import tqdm
import jdatetime
import pandas as pd

# change Date and Time to correct format 

In [16]:
sahamData = pd.read_csv("palayesh.csv")

In [17]:
listDate = [date.split("-") for date in list(sahamData['J-Date'])]

In [18]:
sahamData['year'] = [date[0] for date in listDate]
sahamData['month'] = [date[1] for date in listDate]
sahamData['day'] = [date[2] for date in listDate]

In [19]:
listTime = [date.split(":") for date in list(sahamData['Time'])]

In [20]:
sahamData['hour'] = [date[0] for date in listTime]
sahamData['minute'] = [date[1] for date in listTime]
sahamData['second'] = [date[2] for date in listTime]

In [21]:
listDay = list(sahamData['day'])
listYear = list(sahamData['year'])
listMonth = list(sahamData['month'])

In [22]:
listTarikh = []
for i in tqdm(range(len(list(sahamData['year'])))):
    listTarikh.append(str(jdatetime.date(day = int(listDay[i]) , month = int(listMonth[i]) , year = int(listYear[i])).togregorian()))

100%|██████████| 16887164/16887164 [06:02<00:00, 46534.16it/s]


In [23]:
sahamData['tarikh'] = listTarikh

In [24]:
df  = sahamData

# Change index to Time

In [25]:
df["time"] = pd.to_datetime(sahamData['tarikh'] + " " +
                            df["hour"] + ":" + df["minute"] + ":" + df["second"],
                            format="%Y-%m-%d %H:%M:%S")

# Set the time column as the index
df = df.set_index("time")

In [26]:
df

,J-Date,Time,Volume,Price,year,month,day,hour,minute,second,tarikh
time,,,,,,,,,,,
2020-12-01 12:00:05,1399-09-11,12:00:05,500,119000,1399,09,11,12,00,05,2020-12-01
2020-12-01 12:00:05,1399-09-11,12:00:05,500,119000,1399,09,11,12,00,05,2020-12-01
2020-12-01 12:00:05,1399-09-11,12:00:05,500,119000,1399,09,11,12,00,05,2020-12-01
2020-12-01 12:00:05,1399-09-11,12:00:05,500,119000,1399,09,11,12,00,05,2020-12-01
2020-12-01 12:00:05,1399-09-11,12:00:05,500,119000,1399,09,11,12,00,05,2020-12-01
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-19 12:29:59,1402-01-30,12:29:59,17,184950,1402,01,30,12,29,59,2023-04-19
2023-04-19 12:29:59,1402-01-30,12:29:59,1000,184950,1402,01,30,12,29,59,2023-04-19
2023-04-19 12:29:59,1402-01-30,12:29:59,250,184950,1402,01,30,12,29,59,2023-04-19


# function for making candles

In [27]:
def getCandles(data , timeFrame):
    
    # Resample to one-minute intervals and aggregate using open-high-low-close
    ohlc_dict = {"Price": ["first", "max", "min", "last"] }
    ohlc_dict2 = {"Volume":{"sum"} }

    
    candles = data.resample(timeFrame).agg(ohlc_dict)
    candles2 = data.resample(timeFrame).agg(ohlc_dict2)



    # Flatten multi-level column names
    candles.columns = [f"{col[0]}_{col[1]}" for col in candles.columns]
    candles2.columns = [f"{col[0]}_{col[1]}" for col in candles2.columns]
    candles = pd.concat([candles , candles2] , axis = 1)

    # Calculate the high, low, open, and close prices for each candle
    candles["high"] = candles["Price_max"]
    candles["low"] = candles["Price_min"]
    candles["open"] = candles["Price_first"]
    candles["close"] = candles["Price_last"]
    candles["volume"] = candles["Volume_sum"]

    # Drop any rows with missing values
    candles = candles.dropna()

    # Print the resulting candles
    return candles
    

# make candles of different timeFrames

In [ ]:
listTimeFrames = ['5min','30min' , '1H' , '1D' ]

In [ ]:
for timeFrame in listTimeFrames:
    candles = getCandles( df , timeFrame)
    candles.to_csv(f"palayesh_{timeFrame}.csv")
